In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain
from itertools import combinations
from tqdm import tqdm #visualize long iteration

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
952,tt0220506,13000000,37664855,Halloween: Resurrection,Jamie Lee Curtis|Brad Loree|Busta Rhymes|Bianc...,Rick Rosenthal,Evil finds its way home,Serial Killer Michael Myers is not finished wi...,94,Horror|Thriller,Dimension Films,7/1/2002,4.6,2002
1522,tt0454919,38000000,29907685,Pulse,Kristen Bell|Ian Somerhalder|Christina Milian|...,Jim Sonzero,You are now infected.,When their computer hacker friend accidentally...,90,Horror|Thriller,Dimension Films,8/11/2006,5.1,2006
1092,tt0319061,70000000,122919055,Big Fish,Ewan McGregor|Albert Finney|Billy Crudup|Jessi...,Tim Burton,An adventure as big as life itself.,Throughout his life Edward Bloom has always be...,125,Adventure|Fantasy|Drama,The Zanuck Company|Columbia Pictures Corporati...,12/25/2003,7.4,2003
52,tt0810819,15000000,64191523,The Danish Girl,Eddie Redmayne|Alicia Vikander|Matthias Schoen...,Tom Hooper,Find the courage to be yourself.,When Gerda Wegener asks her husband Einar to f...,120,Drama,Pretty Pictures|Working Title Films|ArtÃ©mis P...,11/27/2015,7.3,2015
555,tt0225071,53000000,29700000,Angel Eyes,Jennifer Lopez|Jim Caviezel|Jeremy Sisto|Terre...,Luis Mandoki,"The deeper you look, the more you will find.",A story about a seemingly unlikely couple who ...,102,Drama|Romance|Thriller,Warner Bros. Pictures|Franchise Pictures|Morga...,5/15/2001,5.6,2001


In [5]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [6]:
answers = {} # создадим словарь для ответов

# структура таблицы
 #0   imdb_id - object - ID
 #1   budget - int64 - бюджет
 #2   revenue - int64 - сборы
 #3   original_title - object - оригинальное название
 #4   cast - object - в ролях
 #5   director - object - режиссер
 #6   tagline - object - слоган
 #7   overview - object - описание фильма
 #8   runtime - int64 - время фильма
 #9   genres - object - жанры
 #10  production_companies - object - издатель
 #11  release_date - object - Дата выпуска
 #12  vote_average - float64 - рейтинг
 #13  release_year - int64 - год выпуска
 #14  profit = revenue - budget -> int64 прибыль = сборы - бюджет
 #15  genres_list - список жанров

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
data['release_date'] = pd.to_datetime(data.release_date)
# сформируем колонку прибыль = сборы - бюджет
data['profit'] = data['revenue'] - data['budget']

# сформируем колонку - список жанров
data['genres_list'] = data['genres'].str.split('|')
data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   imdb_id               1889 non-null   object        
 1   budget                1889 non-null   int64         
 2   revenue               1889 non-null   int64         
 3   original_title        1889 non-null   object        
 4   cast                  1889 non-null   object        
 5   director              1889 non-null   object        
 6   tagline               1889 non-null   object        
 7   overview              1889 non-null   object        
 8   runtime               1889 non-null   int64         
 9   genres                1889 non-null   object        
 10  production_companies  1889 non-null   object        
 11  release_date          1889 non-null   datetime64[ns]
 12  vote_average          1889 non-null   float64       
 13  release_year      

,budget,revenue,runtime,vote_average,release_year,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,2.544506e+09


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [7]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
#answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)' #+
# если ответили верно, можете добавить комментарий со значком "+"

In [8]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,2011-05-11,6.3,2011,641683000,"[Adventure, Action, Fantasy]"


ВАРИАНТ 2

In [9]:
# можно добавлять разные варианты решения
data.query('budget == budget.max()')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,2011-05-11,6.3,2011,641683000,"[Adventure, Action, Fantasy]"


# 2. Какой из фильмов самый длительный (в минутах)?

In [10]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)' #+

In [11]:
data.query('runtime == runtime.max()')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2003-02-21,5.8,2003,-43076064,"[Drama, History, War]"


In [12]:
data.sort_values(by='runtime', ascending=False).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2003-02-21,5.8,2003,-43076064,"[Drama, History, War]"


# 3. Какой из фильмов самый короткий (в минутах)?


In [13]:
answers['3'] = '3. Winnie the Pooh (tt1449283)' #+

In [14]:
data.query('runtime == runtime.min()')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,2011-04-13,6.8,2011,-15540000,"[Animation, Family]"


# 4. Какова средняя длительность фильмов?


In [15]:
answers['4'] = '2. 110' #+

In [16]:
round(data.runtime.mean())


110

# 5. Каково медианное значение длительности фильмов? 

In [17]:
answers['5'] = '1. 107' #+

In [18]:
round(data.runtime.median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [19]:
# лучше код получения столбца profit вынести в Предобработку что в начале

In [20]:
answers['6'] = '5. Avatar (tt0499549)' #+


In [21]:
data.query('profit == profit.max()')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,2009-12-10,7.1,2009,2544505847,"[Action, Adventure, Fantasy, Science Fiction]"


# 7. Какой фильм самый убыточный? 

In [22]:
answers['7'] = '5. The Lone Ranger (tt1210819)' #+


In [23]:
data.query('profit == profit.min()')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,-165710090,"[Action, Adventure, Western]"


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [24]:
answers['8'] = '1. 1478' #+


In [25]:
len(data.query('revenue > budget'))


1478

ВАРИАНТ 2


In [26]:
data.query('profit >= 0').profit.count()


1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [27]:
answers['9'] = '4. The Dark Knight (tt0468569)' #+


In [28]:
data.query('release_year == 2008').\
    sort_values(by='profit', ascending=False).\
    head(1)


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,2008-07-16,8.1,2008,816921825,"[Drama, Action, Crime, Thriller]"


ВАРИАНТ 2


In [29]:
max_profit_2008 = data[data['release_year'] == 2008].profit.max()
data[(data['release_year'] == 2008) & (data['profit'] == max_profit_2008)]


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,2008-07-16,8.1,2008,816921825,"[Drama, Action, Crime, Thriller]"


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [30]:
answers['10'] = '5. The Lone Ranger (tt1210819)' #+


In [31]:
data.query('2012 <= release_year <= 2014').\
    sort_values(by='profit').\
    head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,-165710090,"[Action, Adventure, Western]"


# 11. Какого жанра фильмов больше всего?

In [32]:
answers['11'] = '3. Drama' #+
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [33]:
pd.Series(Counter(chain.from_iterable(data.genres_list))).\
    sort_values().\
    tail(1)


Drama    782
dtype: int64

ВАРИАНТ 2

In [34]:
pd.Series(list(chain.from_iterable(data.genres_list))).\
    value_counts().\
    head(1)


Drama    782
dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [35]:
answers['12'] = '1. Drama' #+


In [36]:
pd.Series\
    (Counter(chain.from_iterable(data.query('profit > 0').genres_list))).\
    sort_values().\
    tail(1)

Drama    560
dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [37]:
answers['13'] = '5. Peter Jackson' #+


In [38]:
directors_list = data.director.str.split('|')
uniq_directors = pd.Series(chain.from_iterable(directors_list)).unique()
directors_counter = Counter()
for director in uniq_directors:
    directors_counter[director] = data[data.director.str.contains(director)].revenue.sum()
directors_counter.most_common(1)

[('Peter Jackson', 6490593685)]

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [39]:
answers['14'] = '3. Robert Rodriguez' #+


In [40]:
data_tmp = data[data.genres.str.contains('Action')]
Counter(chain.from_iterable(data_tmp.director.str.split('|'))).\
    most_common(1)


[('Robert Rodriguez', 9)]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [41]:
answers['15'] = '3. Chris Hemsworth' #+


In [42]:
data_tmp = data[data.release_year == 2012]
cast_list = data_tmp.cast.str.split('|')
uniq_casts = pd.Series(chain.from_iterable(cast_list)).unique()
casts_counter = Counter()
for cast in uniq_casts:
    casts_counter[cast] = data_tmp[data_tmp.cast.str.contains(cast)].revenue.sum()
casts_counter.most_common(1)


[('Chris Hemsworth', 2027450773)]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [43]:
answers['16'] = '3. Matt Damon' #+


In [44]:
data_tmp = data.query('budget > budget.mean()')
Counter(chain.from_iterable(data_tmp.cast.str.split('|'))).\
    most_common(1)


[('Matt Damon', 18)]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [45]:
answers['17'] = '2. Action' #+


In [46]:
data_tmp = data[data.cast.str.contains('Nicolas Cage')]
Counter(chain.from_iterable(data_tmp.genres_list)).\
    most_common(1)


[('Action', 17)]

# 18. Самый убыточный фильм от Paramount Pictures

In [47]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)' #+


In [48]:
data[data.production_companies.str.contains('Paramount Pictures')].\
    query('profit == profit.min()')


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,genres_list
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,2002-07-19,6.0,2002,-64831034,"[Thriller, Drama, History]"


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [49]:
answers['19'] = '5. 2015' #+


In [50]:
data.groupby(by='release_year').revenue.sum().tail(1)


release_year
2015    25449202382
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [51]:
answers['20'] = '1. 2014' #+


In [52]:
data[data.production_companies.str.contains('Warner Bros')].\
    groupby('release_year').profit.sum().\
    sort_values().\
    tail(1)


release_year
2014    2295464519
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [53]:
answers['21'] = '4. Сентябрь' #+


In [54]:
Counter(data.release_date.dt.month).most_common(1)


[(9, 227)]

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [55]:
answers['22'] = '2. 450' #+


In [56]:
data[data.release_date.dt.month.isin([6, 7 ,8])].\
    release_date.\
    count()


450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [57]:
answers['23'] = '5. Peter Jackson'


In [58]:
Counter(\
    chain.from_iterable(\
        data[data.release_date.dt.month.isin([12, 1 ,2])].\
            director.str.split('|'))).\
    most_common(1)


[('Peter Jackson', 7)]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [59]:
answers['24'] = '5. Four By Two Productions'


In [60]:
company_counter = Counter()
for company in chain.from_iterable(data.production_companies.str.split('|')):
    company_counter[company] = \
        data[data.production_companies.str.contains(company)].\
            original_title.str.len().\
            mean()
company_counter.most_common(1)


/Users/serpent/.conda/envs/testgit/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


[('Four By Two Productions', 83.0)]

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [61]:
answers['25'] = '3. Midnight Picture Show'


In [62]:
company_counter = Counter()
for company in chain.from_iterable(data.production_companies.str.split('|')):
    company_counter[company] = \
        data[data.production_companies.str.contains(company)].\
            overview.str.split()\
            .str.len().\
            mean()
company_counter.most_common(1)


[('Midnight Picture Show', 175.0)]

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [63]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'


In [64]:
data.query('vote_average > vote_average.quantile(0.99)').original_title


9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [8]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'


NameError: name 'answers' is not defined

In [7]:
casts_counter = Counter()
for index, row in tqdm(data.iterrows()):
    casts_list = row.cast.split('|')
    for cast_pair in combinations(chain(casts_list),2):
        count_pair = len(\
            data[\
                (data.cast.str.contains(cast_pair[0])) & \
                (data.cast.str.contains(cast_pair[1]))])
        casts_counter[cast_pair] = count_pair
print(casts_counter.most_common(1))


1889it [00:44, 42.73it/s]


[(('Daniel Radcliffe', 'Rupert Grint'), 8)]


ВАРИАНТ 2

# Submission

In [66]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': ''}

In [67]:
# и убедиться что ни чего не пропустил)
len(answers)

27